In [1]:
import secrets, base64, qrcode, time, struct, hmac, hashlib

### Generate a secret key

In [ ]:
def generate_secret(length: int = 20):
    secret_bytes = secrets.token_bytes(length)
    #Convert to base32
    base32 = base64.b32encode(secret_bytes).decode('utf-8')
    return base32

SyntaxError: invalid syntax. Perhaps you forgot a comma? (514069040.py, line 1)